# Prerequisites
본 `ipynb` 은 `Python=3.12` 에서 작성하였습니다. Package dependency 를 해결하기 위해 아래 cell 을 실행해주세요.

## Install Python packages

In [ ]:
%pip -q install -U dotenv openai pandas

## Load environment variables from a .env file
secret 노출을 피하고 notebook 들간의 일관된 환경변수를 설정하기 위해 `dotenv` 을 이용한다.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

# Optimization
OpenAI 는 다양한 gpt-5 매개변수를 제공한다. 이를 통해 성능과 비용을 선택적으로 개선해 나갈 수 있다.

In [ ]:
from openai import OpenAI

# OpenAI 클라이언트를 생성합니다.
client = OpenAI(
    base_url=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

## Verbosity
'재잘거림' 의 정도를 결정한다. 가끔씩 gpt 가 너무 친철하다고 생각할 때 이용해보자.

In [ ]:
import pandas as pd

verbosity = {}
response = {}
for v in ["low", "medium", "high"]: # medium 이 default
    response[v] = client.responses.create(
        model="gpt-5",
        input="내가 오늘 화성에 한 번 가보고 싶은데, 어떻게 갈 수 있을까 ?",
        text={"verbosity": v},
    )
    verbosity[v] = [response[v].usage.output_tokens]

# pandas 로 결과 출력
pd.DataFrame(verbosity)

In [ ]:
# 주의해야 할 점이, max_output_tokens 이 실제 생성되는 tokens 수보다 작으면 output_text 가 리턴되지 않는다.
# 차라리, streaming 으로 받아서 처리하는게 좋다.
response = client.responses.create(
    model="gpt-5",
    input="내가 오늘 화성에 한 번 가보고 싶은데, 어떻게 갈 수 있을까 ?",
    max_output_tokens=1000,
)

print("====== 출력 ======")
for item in response.output:
    if item.type == "message":
        for content in item.content:
            print(content.text)
print("====== 사용량 ======")
print(response.usage.model_dump_json(indent=2))

## Context-Free Grammar (CFG)
출력되는 output 을 regex 와 같은 도구를 통해 문법을 지키도록 할 수 있다.

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input="generate_name 을 호출해서 새로 태어나는 아이의 이름을 지어줘.",
    tools=[
        {
            "type": "custom",
            "name": "generate_name",
            "description": "이름을 지어주는 도구입니다.",
            "format": {
                "type": "grammar",
                "syntax": "regex",
                "definition": "^이[가-힣]{1,2}$",
            }
        }
    ]
)

print("====== 출력 ======")
for item in response.output:
    if item.type == "custom_tool_call":
        print(item.input)
print("====== 사용량 ======")
print(response.usage.model_dump_json(indent=2))

## Minimal Reasoning
추론의 정도를 조절한다. RAG 의 정보가 많지 않거나 task 가 깊은 추론을 요구하지 않는다면, 조절해볼만도 하다.

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input="물에 어떤 불순물이 들어갔는데, 100도가 되도 물이 끓지 않아. 어떤 불순물이 들어간 걸까 ?",
    reasoning={"effort": "high"},    # "minimal", "high"
)

print("====== 출력 ======")
for item in response.output:
    if item.type == "message":
        for content in item.content:
            print(content.text)
print("====== 사용량 ======")
print(response.usage.model_dump_json(indent=2))

# Parallel tool calling
Tool calling 할때 병렬처리 지원한다. Client-side 에서 async / multi-threads 기반으로 운용한다면 보다 빠른 처리가 가능하다.

In [ ]:
tools = [
    {
        "type": "function",
        "name": "get_age",
        "function": {
            "name": "get_age",
            "description": "인물의 나이를 알려주는 함수입니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": { "type": "string" }
                },
                "required": ["name"]
            },
        },
    },
    {
        "type": "function",
        "name": "get_height",
        "function": {
            "name": "get_height",
            "description": "인물의 키를 알려주는 함수입니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": { "type": "string" }
                },
                "required": ["name"]
            },
        },
    },
    {
        "type": "function",
        "name": "get_weight",
        "function": {
            "name": "get_weight",
            "description": "인물의 체중을 알려주는 함수입니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": { "type": "string" }
                },
                "required": ["name"]
            },
        }
    }
]

response = client.responses.create(
    model="gpt-5",
    input="이진호의 나이과 키와 체중을 알려줘.",
    tools=tools,
    parallel_tool_calls=True,  # 병렬로 도구를 호출합니다.
)
print("====== 출력 (gpt-5 병렬 on)======")
print(response.model_dump_json(indent=2, exclude_none=True))
response = client.responses.create(
    model="gpt-5",
    input="이진호의 나이과 키와 체중을 알려줘.",
    tools=tools,
    parallel_tool_calls=False,  # 병렬로 도구를 호출합니다.
)
print("====== 출력 (gpt-5 병렬 off)======")
print(response.model_dump_json(indent=2, exclude_none=True))